In [ ]:
%run ./bootstrap_script

In [ ]:
import os
import requests
import json
from requests.auth import HTTPBasicAuth

In [ ]:
# Parameter Configuration

dbutils.widgets.text("Root_S3_Bucket_for_Workspace", "new-databricks-development-ws-bkt-767397898349")
dbutils.widgets.text("IAM_ARN_for_Cred_Config", "arn:aws:iam::767397898349:role/new-databricks-development-cred-config-iam-role")
dbutils.widgets.text("Workspace_Name", "new-databricks-development-ws-767397898349")

In [ ]:
class DBX_UTILITY:
    def __init__(self):
        # Define the endpoint and credentials
        self.token_base_url = f"{os.getenv('DATABRICKS_HOST')}/oidc/accounts/{os.getenv('DATABRICKS_ACCOUNT_ID')}"
        self.base_url = f"{os.getenv('DATABRICKS_HOST')}/api/2.0/accounts/{os.getenv('DATABRICKS_ACCOUNT_ID')}"
        self.client_id = f"{os.getenv('DATABRICKS_CLIENT_ID')}"
        self.client_secret = f"{os.getenv('DATABRICKS_CLIENT_SECRET')}"
        self.region = f"{os.getenv('DATABRICKS_REGION')}"
        self.root_s3_bucket_for_workspace = dbutils.widgets.get("Root_S3_Bucket_for_Workspace")
        self.iam_arn_for_cred_config = dbutils.widgets.get("IAM_ARN_for_Cred_Config")
        self.workspace_name = dbutils.widgets.get("Workspace_Name")

    def refreshToken(self):
        # Define the payload
        data = {
            'grant_type': 'client_credentials',
            'scope': 'all-apis'
        }

        # Make the POST request
        response = requests.post(f"{self.token_base_url}/v1/token", auth=HTTPBasicAuth(f"{self.client_id}", f"{self.client_secret}"), data=data)

        if response.status_code == 200:
            return response.json().get('access_token')
        else:
            response.raise_for_status()
    
    def getMetastoreId(self, bearer_token):
        # Define the headers
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }

        # Make the GET request
        response = requests.get(f"{self.base_url}/metastores", headers=headers)

        if response.status_code == 200:
            metastore_arr = response.json().get('metastores')
            return next((elem['metastore_id'] for elem in metastore_arr if elem['region'] == self.region), None)
        else:
            response.raise_for_status()
    
    def createStorageConfiguration(self, bearer_token):
        # Define the headers
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }

        # Define the payload
        data = {
            "storage_configuration_name": "new_strg_cnfgrtn",
            "root_bucket_info": {
                "bucket_name": f"{self.root_s3_bucket_for_workspace}"
            }
        }

        # Make the POST request
        response = requests.post(
            f"{self.base_url}/storage-configurations",
            headers=headers,
            json=data
        )

        if response.status_code == 201:
            return response.json().get('storage_configuration_id')
        else:
            print("Error response:", response.text)
            response.raise_for_status()
    
    def listStorageConfiguration(self, bearer_token):
        # Define the headers
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }

        # Make the GET request
        response = requests.get(f"{self.base_url}/storage-configurations", headers=headers)
        
        strgCredList = [elem['storage_configuration_id'] for elem in response.json()]

        if response.status_code == 200:
            return strgCredList
        else:
            response.raise_for_status()

    def deleteStorageConfiguration(self, bearer_token, strg_config_id):
        # Define the headers
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }

        # Make the GET request
        response = requests.delete(f"{self.base_url}/storage-configurations/{strg_config_id}", headers=headers)
        
        # strgCredList = [elem['storage_configuration_id'] for elem in response.json()]

        if response.status_code == 200:
            return response.text
        else:
            response.raise_for_status()

    def createCredentialConfiguration(self, bearer_token):
        # Define the headers
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }

        # Define the payload
        data = {
            "credentials_name": "new_cred_config",
            "aws_credentials": {
                "sts_role": {
                    "role_arn": f"{self.iam_arn_for_cred_config}"
                }
            }
        }

        # Make the POST request
        response = requests.post(
            f"{self.base_url}/credentials",
            headers=headers,
            json=data
        )

        if response.status_code == 201:
            return response.json().get('credentials_id')
        else:
            print("Error response:", response.text)
            response.raise_for_status()

    def createWorkspace(self, bearer_token, cred_id, strg_config_id):
        # Define the headers
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }

        # Define the payload
        data = {
            "workspace_name": f"{self.workspace_name}",
            "aws_region": f"{self.region}",
            "credentials_id": f"{cred_id}",
            "storage_configuration_id": f"{strg_config_id}",
            "pricing_tier": "PREMIUM"
        }

        # Make the POST request
        response = requests.post(
            f"{self.base_url}/workspaces",
            headers=headers,
            json=data
        )

        if response.status_code == 201:
            return response.json().get('workspace_id')
        else:
            print("Error response:", response.text)
            response.raise_for_status()

utility = DBX_UTILITY()

In [ ]:
# Bearer Token Creation
token = utility.refreshToken()

In [ ]:
# Get Metastore ID
# if token:
#     metastore_id = utility.getMetastoreId(token)

In [ ]:
# Delete All Storage Credentials
# if token:
#     strConfigList = utility.listStorageConfiguration(token)
#     if strConfigList:
#         for strgCred in strConfigList:
#             try:
#                 print("Deleted: " + str(utility.deleteStorageConfiguration(token, strgCred)))
#             except:
#                 print("Error: " + str(strgCred))

In [ ]:
# Storage Config Creation
if token:
    storage_config_id = utility.createStorageConfiguration(token)
    print(storage_config_id)

In [ ]:
# Cred Config Creation
if token:
    cred_config_id = utility.createCredentialConfiguration(token)
    print(cred_config_id)

In [ ]:
# Worspace Creation
if token and cred_config_id and storage_config_id:
    workspace_id = utility.createWorkspace(token, cred_config_id, storage_config_id)

In [ ]:
print(workspace_id)